In [1]:
import os
os.chdir('/home/deep09/project_vlad/parallel-tempering/')
import gc
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle as arys_shuffle

from simulator.simulator import Simulator
from simulator.models.cifar10_models import resnet
from simulator import simulator_utils as s_utils
'''
# train a single replica
steps = {'train_step': 176,
         'test_step': 352,
         'valid_step': 352}
train_resnet.train(182, steps)
'''



"\n# train a single replica\nsteps = {'train_step': 176,\n         'test_step': 352,\n         'valid_step': 352}\ntrain_resnet.train(182, steps)\n"

In [2]:
# 0.005 0.0005
# coeff lower ==> accepts higher
params = dict(
        n_epochs=[182, 182, 182, 182, 182, 182],
        learning_rate=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        n_replicas=[6, 8, 8, 6, 8, 8],
        noise_type=['learning_rate_momentum', 'learning_rate_momentum', 'learning_rate_momentum', 'learning_rate_momentum', 'learning_rate_momentum', 'learning_rate_momentum'],
        func_name=['resnet20', 'resnet32', 'resnet44', 'resnet20', 'resnet32', 'resnet44'],
        dataset_name=['cifar', 'cifar', 'cifar', 'cifar', 'cifar', 'cifar'],
        loss_func_name=['cross_entropy', 'cross_entropy', 'cross_entropy', 'cross_entropy', 'cross_entropy', 'cross_entropy'],
        train_data_size=[45000, 45000, 45000, 45000, 45000, 45000],
        model=[None, None, None, None, None, None],
        n_simulations=[1, 1, 1, 1, 1, 1],
        description=['test', 'test', 'test', 'test', 'test', 'test'],
        separation_ratio=[0, 0, 0, 0, 0, 0],
        beta_0=[0.05, 0.02, 0.02, 0.05, 0.03, 0.02, ],
        beta_n=[0.005,  0.0005, 0.0001, 0.005,  0.001, 0.0001,],
        proba_coeff=[0.1, 0.02, 0.005, 0.1, 0.02, 0.005,],
        burn_in_period=[32000, 32000, 32000, 32000, 32000, 32000, ],
        batch_size=[128, 128, 128, 128, 128, 128],
        swap_step=[200, 200, 200, 200, 200, 200],
        mode=[None, None, None, None, None, None],
        scheduled_noise=[True, True, True, True, True, True],
        resnet_size=[20, 32, 44, 20, 32, 44])

test_step = 352
n_experiments = 6
timer = s_utils.Timer()
timer.start_timer()
total_sims = np.sum(params['n_simulations'])

assert all(len(v) >= n_experiments for k, v in params.items())



In [ ]:
from simulator.read_datasets import _create_cifar_data_or_get_existing_resnet
x_train, y_train, x_test, y_test, x_valid, y_valid = (
        _create_cifar_data_or_get_existing_resnet())

for i in range(n_experiments):
    #if i <= 2: continue
    simulation_num = 0



    noise_list = list(np.linspace(params['beta_0'][i], params['beta_n'][i], params['n_replicas'][i]))
    if params['scheduled_noise'][i]:
        scheduled_noise = {32000: noise_list,
                           1: [0.1 for _ in range(params['n_replicas'][i])]}
    #noise_list = [0.018, 0.22]
    name = s_utils.generate_experiment_name(model_name=params['func_name'][i],
                                            dataset_name=params['dataset_name'][i],
                                            separation_ratio=params['separation_ratio'][i],
                                            n_replicas=params['n_replicas'][i],
                                            beta_0=params['beta_0'][i],
                                            beta_n=params['beta_n'][i],
                                            loss_func_name=params['loss_func_name'][i],
                                            swap_step=params['swap_step'][i],
                                            burn_in_period=params['burn_in_period'][i],
                                            learning_rate=params['learning_rate'][i],
                                            n_epochs=params['n_epochs'][i],
                                            noise_type=params['noise_type'][i],
                                            batch_size=params['batch_size'][i],
                                            proba_coeff=params['proba_coeff'][i],
                                            train_data_size=params['train_data_size'][i],
                                            mode=params['mode'][i])
    ensembles = resnet(tf.Graph(), params['n_replicas'][i], params['resnet_size'][i])
    sim = Simulator(model=params['model'][i],
                    learning_rate=params['learning_rate'][i],
                    noise_list=noise_list,
                    noise_type=params['noise_type'][i],
                    batch_size=params['batch_size'][i],
                    n_epochs=params['n_epochs'][i],
                    name=name,
                    ensembles=ensembles,
                    burn_in_period=params['burn_in_period'][i],
                    swap_step=params['swap_step'][i],
                    separation_ratio=params['separation_ratio'][i],
                    n_simulations=params['n_simulations'][i],
                    scheduled_noise=scheduled_noise,
                    test_step=test_step,
                    loss_func_name=params['loss_func_name'][i],
                    proba_coeff=params['proba_coeff'][i],
                    mode=params['mode'][i])
    
    for sim_num in range(params['n_simulations'][i]):
        simulation_num += 1
        '''
        train_data, train_labels, test_data, test_labels, valid_data, valid_labels = (
                                    read_datasets.get_cifar10_data())
        
        '''
        
        
        print(name, str(simulation_num) + '/' + str(total_sims))
        print('train-data-shape: {}'.format(x_train.shape))
        sim.train(train_data_size=45000,
                  train_data=x_train,
                  train_labels=y_train,
                  validation_data=x_valid,
                  validation_labels=y_valid,
                  test_data=x_test,
                  test_labels=y_test)
        del sim
        del ensembles
        gc.collect()
        print()
        print(str(simulation_num) + '/' + str(n_experiments), ', time took:', timer.elapsed_time())

    

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


W0520 00:01:44.900784 140150092326656 tf_logging.py:125] From /home/deep09/project_vlad/parallel-tempering/simulator/models/resnet/resnet_v2.py:33: UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


resnet20_cifar_0_45000_6_0.05_crossentropy_200_32000_0.1_182_128_learningratemomentum_0.1_0.005_None_v3 1/6
train-data-shape: (45000, 32, 32, 3)
[epoch:181]|[step:64065]|[0.100, 0.095, 0.096, 0.098, 0.102, 0.105]|[accept:0.931]
1/6 , time took: 161
resnet32_cifar_0_45000_8_0.03_crossentropy_200_32000_0.1_182_128_learningratemomentum_0.08_0.001_None_v3 1/6
train-data-shape: (45000, 32, 32, 3)
[epoch:181]|[step:64065]|[0.090, 0.089, 0.088, 0.093, 0.093, 0.089, 0.090, 0.092]|[accept:0.875]
1/6 , time took: 321
resnet44_cifar_0_45000_8_0.02_crossentropy_200_32000_0.1_182_128_learningratemomentum_0.06_0.0005_None_v3 1/6
train-data-shape: (45000, 32, 32, 3)
[epoch:164]|[step:57998]|[0.086, 0.089, 0.086, 0.082, 0.087, 0.085, 0.083, 0.087]|[accept:0.891]